# Test Fly Abdomens Segmentation

This notebook is used to test separately each step used in the detection and segmentation of the surface of the abdomen in an image stack.

In [1]:
# import packages 
from skimage import io, transform
import numpy as np
import open3d as o3d
import copy
import napari
import time
import os
from skimage import morphology
from skimage.measure import label, regionprops, block_reduce
from scipy import stats, ndimage
import matplotlib.pyplot as plt
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from src.preprocess import segmentation_with_optimized_thresh, image_padding
from src.registration import image_to_pcd, pcd_to_image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Read the image, only fluorescence channel is required:
# reference_fly_filename = "../../data_2/01_raw/C1-D3_En_male_03_20220805.tif"
reference_fly_filename = "../../data_2/01_raw/C1-D0_En_pn_male_06_20220708.tif"

image = io.imread(reference_fly_filename)

# Downscaling:
downscaling =  (1,2.5,2.5)
new_image_shape = [int(image.shape[i]/downscaling[i]) for i in range(3)]
image_downscaled = transform.resize(image, new_image_shape, preserve_range = True, anti_aliasing=True)
#viewer = napari.view_image(image_downscaled)

### Thresholding:

In [3]:
t0 = time.time()
# Segment:
Thresholded_Image = segmentation_with_optimized_thresh(image_downscaled, fraction_range = [0.05, 0.06])
t1 = time.time()
viewer = napari.view_image(Thresholded_Image)
# print running time:

print("Preprocessing of the image took: ", t1-t0, "seconds")

Preprocessing of the image took:  0.2934141159057617 seconds


INFO - 2023-04-20 18:17:51,900 - acceleratesupport - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


## Clean up the segmented volume with morphological transformations:

In [4]:
t0 = time.time()

# Padding:
Thresholded_Image  = image_padding(Thresholded_Image)
Image_padded = image_padding(image_downscaled)

# Clean up the segmentation with morphological transformations:
closing_r1 = 4

# Dilate and erode again to fill small holes:
filled = morphology.closing(Thresholded_Image, morphology.ball(closing_r1))

label_image = label(filled)
rp = regionprops(label_image)
size = max([i.area for i in rp])

biggest_objects = morphology.remove_small_objects(label_image, min_size=size/100)>0
Thresholded_Image = biggest_objects

# make the segmented volume thin by looking for the maxima in the signal
from scipy.signal import find_peaks
def local_maxima_z(image, dist):
    # maxima along z
    result = np.zeros(image.shape)
    
    for i in range(image.shape[1]):
        for j in range(image.shape[2]):
            peaks, _ = find_peaks(image[:,i,j], distance = dist)
            for p in peaks:
                result[p,i,j] = 1
    
    return result

def local_maxima_x(image, dist):
    # maxima along x
    result = np.zeros(image.shape)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[2]):
            peaks, _ = find_peaks(image[i,:,j], distance = dist)
            for p in peaks:
                result[i,p,j] = 1
    
    return result

def local_maxima_y(image, dist):
    # maxima along y
    result = np.zeros(image.shape)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            peaks, _ = find_peaks(image[i,j,:], distance = dist)
            for p in peaks:
                result[i,j,p] = 1
    
    return result

Maxima_z = local_maxima_z(Thresholded_Image*Image_padded, 100)
#Maxima_y = local_maxima_y(Thresholded_Image*Image_padded, 50)
#Maxima_x = local_maxima_x(Thresholded_Image*Image_padded, 50)
Thresholded_Image = Maxima_z#(Maxima_z+Maxima_y+Maxima_x) > 0

t1 = time.time()
viewer = napari.view_image(Thresholded_Image)
# print running time:

print("Preprocessing of the image took: ", t1-t0, "seconds")

Preprocessing of the image took:  8.856957912445068 seconds


## Transform the segmented volume in a point cloud object:

In [5]:
def custom_draw(pcd):
    # The following code achieves the same effect as:
    # o3d.visualization.draw_geometries([pcd])
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    
def custom_draw_w_mesh(pcd, mesh):
    # The following code achieves the same effect as:
    # o3d.visualization.draw_geometries([pcd])
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    opt = vis.get_render_option()
    opt.background_color = np.asarray([0, 0, 0])
    vis.add_geometry(pcd)
    vis.add_geometry(mesh)
    vis.run()
    vis.destroy_window()
    
def draw_pcd(pcd):
    pcd_temp = copy.deepcopy(pcd)
    pcd_temp.paint_uniform_color([1, 0.706, 0])
    o3d.visualization.draw_geometries([pcd_temp])
    

def skeletonize_on_slices(image_3d):
    
    result = np.zeros(image_3d.shape)
    
    for i in range(image_3d.shape[1]):
        image = image_3d[:,i,:]
        skeleton = morphology.skeletonize(image)
        result[:,i,:] = skeleton

    return result
    

# Create a pcd from the thresholded image
pcd, pcd_values = image_to_pcd(Thresholded_Image)
custom_draw(pcd)

# Remove outliers from the pcd, based on n_neighbrours within a radius:
uni_down_pcd = pcd.uniform_down_sample(every_k_points=3)
custom_draw(uni_down_pcd)
cleaned_pcd = uni_down_pcd
#cleaned_pcd, ind = uni_down_pcd.remove_radius_outlier(nb_points=4, radius=4)
#cleaned_pcd, ind = cleaned_pcd.remove_radius_outlier(nb_points=4, radius=4)
#cleaned_pcd, ind = cleaned_pcd.remove_radius_outlier(nb_points=4, radius=3)

# downsampling
cleaned_pcd = cleaned_pcd.voxel_down_sample(voxel_size=5)
custom_draw(cleaned_pcd)

## Fit a mesh through the point cloud to close potential holes:

In [6]:
# Create a mesh that fits through the cleaned points to fill potential holes:
cleaned_pcd.estimate_normals()
cleaned_pcd.orient_normals_consistent_tangent_plane(k=30)

#flip normals:
normals = -np.asarray(cleaned_pcd.normals)
cleaned_pcd.normals = o3d.utility.Vector3dVector(normals)

#visualize normals:
#o3d.visualization.draw_geometries([cleaned_pcd], point_show_normal=True)

radii = [30]
ball_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(cleaned_pcd, o3d.utility.DoubleVector(radii))
bbox = cleaned_pcd.get_axis_aligned_bounding_box()
ball_mesh_crop = ball_mesh.crop(bbox)
ball_mesh_crop.paint_uniform_color([1,1,1])

custom_draw_w_mesh(cleaned_pcd, ball_mesh_crop)
resampled_pcd = ball_mesh_crop.sample_points_uniformly(number_of_points = 50000)
custom_draw(resampled_pcd)

final_pcd = o3d.geometry.PointCloud()
final_pcd.points = o3d.utility.Vector3dVector( np.concatenate((pcd.points, resampled_pcd.points), axis=0) )

custom_draw(final_pcd)

## Create a mask from the mesh and apply it to the original image stack:

In [7]:
final_pcd_values = np.ones(np.asarray(final_pcd.points).shape[0])
final_image = pcd_to_image(final_pcd, final_pcd_values, Thresholded_Image.shape)
final_image = morphology.dilation(final_image, morphology.ball(8))
viewer = napari.view_image(final_image*Image_padded)

## Save the masked image

In [17]:
# Save it as reference
from tifffile import imsave
imsave("../../data_2/References_and_masks/C1_Reference_iso.tiff", (final_image*Image_padded).astype(np.uint16))